<img style="float: left;" src="../images/fanniemae.png">
<br><br><br><br><br><br>
______

# Mortgage Loan Default Classifier
____________
____________

## Problem Statement:
_____________
Fannie Mae, or more specifically the Federal National Mortgage Association (FNMA), is a government sponsored entity whose primary goal is to raise home ownership and affordable housing levels.  Fannie Mae attempts to accomplish this in essence by purchasing mortgage loans within certain parameters from mortgage lenders.  In turn, mortgage lenders are provided cash flow to issue additional mortgages.<br>

The cause of the Financial Crisis of 2008 can in part be drawn back to the purchase of mortgage loans with an actual probability of default that were higher than assumed.  By creating a classification model that will predict whether a mortgage loan will default based on pre-purchase characteristics, Fannie Mae may better avoid high risk mortgage loans.  The model will be evaluated based on Accuracy and False Negative Rate.  In this particular case, the "positive" class will be loans that default therefore, we will seek to minimize the False Negative Rate while maximizing Accuracy.

## Engineered Features, Production Model, and Conclusion
___________
In order to render a better accuracy and false negative rate interaction features and a neural network will be introduced.  Correlation between continuous features and whether a loan default were minimal.  By using PolynomialFeatures, the model can investgate whether the interaction between these features are of more importance than the feature alone.  Additional, a neural network wil be introduced to increase accuracy and decrease the false negative rate.  A logistic regressor has an advantage of being able to indentify key features that have the greatest influence in determining whether a loan will default.  However, as evidenced by the base model, it may not be able to reach the accuracy needed to be a successful model.  A neural network does not have the interpretability inherent in the logistic model however, it has been shown to highly accurate because of their ability to generalise and respond to unexpected patterns. The high degree of inaction between features in this dataset may make a neural network the right choice in this situation.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures 
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/complete2011q1.csv')
df.head()

,LOAN IDENTIFIER,ORIGINATION CHANNEL,SELLER NAME,ORIGINAL INTEREST RATE,ORIGINAL UPB,ORIGINAL LOAN TERM,ORIGINAL LOAN-TO-VALUE (LTV),ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),NUMBER OF BORROWERS,ORIGINAL DEBT TO INCOME RATIO,...,LOAN PURPOSE,PROPERTY TYPE,NUMBER OF UNITS,OCCUPANCY TYPE,PROPERTY STATE,PRODUCT TYPE,RELOCATION MORTGAGE INDICATOR,DEFAULT,MI,MIN CREDIT SCORE
0,100000841305,C,"CITIMORTGAGE, INC.",4.125,124000,360,79,79.0,1.0,28.0,...,R,SF,1,P,TX,FRM,N,0,0.0,792.0
1,100001889356,R,OTHER,4.625,115000,240,68,68.0,1.0,34.0,...,C,SF,1,P,IL,FRM,N,0,0.0,705.0
2,100006453372,C,"BANK OF AMERICA, N.A.",4.375,175000,360,52,52.0,2.0,29.0,...,C,PU,1,S,AZ,FRM,N,0,0.0,776.0
3,100010656545,C,"BANK OF AMERICA, N.A.",4.375,365000,360,59,59.0,3.0,40.0,...,C,PU,1,P,IL,FRM,N,0,0.0,797.0
4,100010758624,R,"CITIMORTGAGE, INC.",3.875,69000,120,28,28.0,1.0,32.0,...,C,SF,1,P,SC,FRM,N,0,0.0,785.0


In [3]:
continuous_features = ['ORIGINAL INTEREST RATE', 'ORIGINAL UPB', 'ORIGINAL LOAN-TO-VALUE (LTV)', 
                       'ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)', 'ORIGINAL DEBT TO INCOME RATIO',
                       'MIN CREDIT SCORE']

In [4]:
poly = PolynomialFeatures(degree=3, include_bias=False)
df_poly = pd.DataFrame(poly.fit_transform(df[continuous_features]), columns=poly.get_feature_names(continuous_features))
df_poly = df_poly.drop(columns=continuous_features)
df_poly = pd.concat([df, df_poly], axis=1)
df_poly.head()

C:\Users\bdmir\Anaconda3\envs\dsi\lib\site-packages\sklearn\preprocessing\data.py:1263: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for c in combinations)


,LOAN IDENTIFIER,ORIGINATION CHANNEL,SELLER NAME,ORIGINAL INTEREST RATE,ORIGINAL UPB,ORIGINAL LOAN TERM,ORIGINAL LOAN-TO-VALUE (LTV),ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),NUMBER OF BORROWERS,ORIGINAL DEBT TO INCOME RATIO,...,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)^3,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)^2 ORIGINAL DEBT TO INCOME RATIO,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)^2 MIN CREDIT SCORE,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV) ORIGINAL DEBT TO INCOME RATIO^2,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV) ORIGINAL DEBT TO INCOME RATIO MIN CREDIT SCORE,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV) MIN CREDIT SCORE^2,ORIGINAL DEBT TO INCOME RATIO^3,ORIGINAL DEBT TO INCOME RATIO^2 MIN CREDIT SCORE,ORIGINAL DEBT TO INCOME RATIO MIN CREDIT SCORE^2,MIN CREDIT SCORE^3
0,100000841305,C,"CITIMORTGAGE, INC.",4.125,124000,360,79,79.0,1.0,28.0,...,493039.0,174748.0,4942872.0,61936.0,1751904.0,49553856.0,21952.0,620928.0,17563392.0,496793088.0
1,100001889356,R,OTHER,4.625,115000,240,68,68.0,1.0,34.0,...,314432.0,157216.0,3259920.0,78608.0,1629960.0,33797700.0,39304.0,814980.0,16898850.0,350402625.0
2,100006453372,C,"BANK OF AMERICA, N.A.",4.375,175000,360,52,52.0,2.0,29.0,...,140608.0,78416.0,2098304.0,43732.0,1170208.0,31313152.0,24389.0,652616.0,17463104.0,467288576.0
3,100010656545,C,"BANK OF AMERICA, N.A.",4.375,365000,360,59,59.0,3.0,40.0,...,205379.0,139240.0,2774357.0,94400.0,1880920.0,37477331.0,64000.0,1275200.0,25408360.0,506261573.0
4,100010758624,R,"CITIMORTGAGE, INC.",3.875,69000,120,28,28.0,1.0,32.0,...,21952.0,25088.0,615440.0,28672.0,703360.0,17254300.0,32768.0,803840.0,19719200.0,483736625.0


In [5]:
categorical_features = ['ORIGINATION CHANNEL', 'SELLER NAME', 'FIRST TIME HOME BUYER INDICATOR', 'LOAN PURPOSE', 
                        'PROPERTY TYPE', 'OCCUPANCY TYPE', 'PROPERTY STATE', 'PRODUCT TYPE', 'RELOCATION MORTGAGE INDICATOR']

In [6]:
df_poly = pd.get_dummies(df_poly, columns=categorical_features, drop_first=True)
df_poly.shape

(504559, 169)

### Target number of observations  = features ^ 2
- 112^2 = 12,544  

Need to upsample minority and downsample majority

In [7]:
df['DEFAULT'].value_counts()

0    502998
1      1561
Name: DEFAULT, dtype: int64

In [8]:
# balance targets
# split classes
df_maj = df_poly[df_poly['DEFAULT'] == 0]
df_min = df_poly[df_poly['DEFAULT'] == 1]

# upsample minority
df_min_resample = resample(df_min,
                           replace=True,
                           n_samples=int(len(df_poly.columns)**2.05),
                           random_state=42)

# downsample majority
df_maj_resample = resample(df_maj, 
                           replace=False,    
                           n_samples=df_min_resample.shape[0],
                           random_state=42)             


# concat downsample and minority
df_resample = pd.concat([df_maj_resample, df_min_resample])
 
# Display new class counts
df_resample['DEFAULT'].value_counts()

1    36912
0    36912
Name: DEFAULT, dtype: int64

In [9]:
# train test split
X = df_resample.drop(columns=['DEFAULT'])
y = df_resample['DEFAULT']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    shuffle=True,
                                                    random_state=42)

In [10]:
ss = StandardScaler()
ss.fit(X_train, y_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

## Logistic Classifier

In [11]:
params = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 0.2, 0.3, 0.4]
}

grid_log_cv = GridSearchCV(LogisticRegression(), params, cv=5, return_train_score=True, n_jobs=-1)
grid_log_cv.fit(X_train_sc, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [12]:
grid_log_cv.best_params_

{'C': 0.4, 'penalty': 'l1'}

In [13]:
grid_log_cv.best_score_

0.7805591677503251

In [14]:
log_opt = grid_log_cv.best_estimator_

In [15]:
log_opt.score(X_test_sc, y_test)

0.7816970091027308

In [40]:
features = pd.DataFrame(log_opt.coef_, index=['Coefs']).T

In [41]:
features['Features'] = X.columns
features['DefaultProb'] = features['Coefs'].map(lambda x: np.exp(x))

In [42]:
pd.set_option('display.max_colwidth', -1)

In [43]:
features.sort_values(by='Coefs').head(10)

,Coefs,Features,DefaultProb
53,-2.104007,ORIGINAL UPB^3,0.121967
42,-1.620308,ORIGINAL INTEREST RATE ORIGINAL UPB MIN CREDIT SCORE,0.197838
44,-1.514897,ORIGINAL INTEREST RATE ORIGINAL LOAN-TO-VALUE (LTV) ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),0.219831
28,-1.388698,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV) MIN CREDIT SCORE,0.249400
2,-1.259733,ORIGINAL UPB,0.283730
55,-1.251780,ORIGINAL UPB^2 ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),0.285995
10,-0.902360,MIN CREDIT SCORE,0.405611
56,-0.804111,ORIGINAL UPB^2 ORIGINAL DEBT TO INCOME RATIO,0.447485
21,-0.700083,ORIGINAL UPB MIN CREDIT SCORE,0.496544
20,-0.670695,ORIGINAL UPB ORIGINAL DEBT TO INCOME RATIO,0.511353


In [44]:
features.sort_values(by='Coefs').tail(10)

,Coefs,Features,DefaultProb
85,0.709059,ORIGINAL DEBT TO INCOME RATIO^2 MIN CREDIT SCORE,2.032078
40,0.772726,ORIGINAL INTEREST RATE ORIGINAL UPB ORIGINAL COMBINED LOAN-TO-VALUE (CLTV),2.165661
49,0.947573,ORIGINAL INTEREST RATE ORIGINAL COMBINED LOAN-TO-VALUE (CLTV) MIN CREDIT SCORE,2.579441
80,0.979216,ORIGINAL COMBINED LOAN-TO-VALUE (CLTV)^2 MIN CREDIT SCORE,2.662368
87,0.984795,MIN CREDIT SCORE^3,2.677263
13,1.023202,ORIGINAL INTEREST RATE ORIGINAL LOAN-TO-VALUE (LTV),2.782088
41,1.115561,ORIGINAL INTEREST RATE ORIGINAL UPB ORIGINAL DEBT TO INCOME RATIO,3.051279
46,1.155019,ORIGINAL INTEREST RATE ORIGINAL LOAN-TO-VALUE (LTV) MIN CREDIT SCORE,3.174084
57,2.674859,ORIGINAL UPB^2 MIN CREDIT SCORE,14.510310
17,3.846629,ORIGINAL UPB^2,46.834939


In [20]:
y_pred = log_opt.predict(X_test_sc)

In [21]:
def false_negative_rate(fn, tp):
    return fn/(tp+fn)

def accuracy(tn, tp, y_test):
    return (tn + tp)/len(y_test)

In [22]:
confusion_matrix(y_test, y_pred)

array([[6967, 2261],
       [1768, 7460]], dtype=int64)

In [23]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [24]:
false_negative_rate(fn, tp)

0.1915908105765063

In [25]:
accuracy(tn, tp, y_test)

0.7816970091027308

In [26]:
df_pred = pd.DataFrame(log_opt.predict_proba(X_test_sc), columns=['NO DEFAULT', 'DEFAULT'])
df_pred['y_pred'] = y_pred
df_pred['y_pred_adj'] = df_pred['DEFAULT'].map(lambda x: 1 if x >= 0.05 else 0)
df_pred.head()

,NO DEFAULT,DEFAULT,y_pred,y_pred_adj
0,0.435455,0.564545,1,1
1,0.165299,0.834701,1,1
2,0.297912,0.702088,1,1
3,0.804277,0.195723,0,1
4,0.054671,0.945329,1,1


In [27]:
confusion_matrix(y_test, df_pred['y_pred_adj'])

array([[1481, 7747],
       [  42, 9186]], dtype=int64)

In [28]:
tn, fp, fn, tp = confusion_matrix(y_test, df_pred['y_pred_adj']).ravel()

In [29]:
false_negative_rate(fn, tp)

0.004551365409622887

In [30]:
accuracy(tn, tp, y_test)

0.5779692241005635

__Insight:__
The interaction features that were engineered increased the accuracy of the logistic model from 0.72 to 0.78.  However, an acceptable false negative rate is not obtainable without a huge loss of accurary and Non-Default misclassification.

## Neural Network

In [31]:
num = X_train_sc.shape[1]

nn = Sequential()

nn.add(Dense(num, activation='relu', input_dim=num))
nn.add(Dropout(0.1))
nn.add(Dense(int(num/2), activation='relu'))
nn.add(Dense(1, activation='sigmoid'))

nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=2)

nn_result = nn.fit(X_train_sc, y_train, epochs=100, validation_data=(X_test_sc, y_test), callbacks=[early])

Train on 55368 samples, validate on 18456 samples
Epoch 1/100
55368/55368 [==============================] - 4s 64us/step - loss: 0.4349 - acc: 0.8020 - val_loss: 0.3578 - val_acc: 0.8486
Epoch 2/100
55368/55368 [==============================] - 3s 58us/step - loss: 0.3264 - acc: 0.8644 - val_loss: 0.2761 - val_acc: 0.8928
Epoch 3/100
55368/55368 [==============================] - 3s 59us/step - loss: 0.2530 - acc: 0.9021 - val_loss: 0.2153 - val_acc: 0.9263
Epoch 4/100
55368/55368 [==============================] - 3s 57us/step - loss: 0.2049 - acc: 0.9238 - val_loss: 0.1771 - val_acc: 0.9360
Epoch 5/100
55368/55368 [==============================] - 3s 57us/step - loss: 0.1760 - acc: 0.9358 - val_loss: 0.1685 - val_acc: 0.9399
Epoch 6/100
55368/55368 [==============================] - 3s 59us/step - loss: 0.1521 - acc: 0.9457 - val_loss: 0.1613 - val_acc: 0.9423
Epoch 7/100
55368/55368 [==============================] - 3s 58us/step - loss: 0.1329 - acc: 0.9529 - val_loss: 0.1336 - 

In [32]:
df_pred_nn = pd.DataFrame(nn.predict_proba(X_test_sc), columns=['DEFAULT'])
df_pred_nn['y_pred'] = nn.predict_classes(X_test_sc)
df_pred_nn['y_pred_adj'] = df_pred_nn['DEFAULT'].map(lambda x: 1 if x >= 0.3 else 0)
df_pred_nn.head()

,DEFAULT,y_pred,y_pred_adj
0,9.970945e-01,1,1
1,7.283272e-01,1,1
2,9.998760e-01,1,1
3,4.637454e-09,0,0
4,8.673341e-01,1,1


In [33]:
confusion_matrix(y_test, df_pred_nn['y_pred_adj'])

array([[8329,  899],
       [  14, 9214]], dtype=int64)

In [34]:
tn, fp, fn, tp = confusion_matrix(y_test, df_pred_nn['y_pred_adj']).ravel()

In [35]:
false_negative_rate(fn, tp)

0.001517121803207629

In [36]:
accuracy(tn, tp, y_test)

0.9505309926311226

## Conclusion
The neural network model with an adjusted threshold of 0.3 is able to obtain 0.90 accuracy and a false negative rate of 0.074% which far surpasses the benchmark of 0.309%.  For reference, Fannie Mae would be able to avoid 235 defaulted loans using this model.  Using an average of $50,000 loss per default, this would equate to a loss avoidance of \$11,750,000 per 100,000 loans.